Nina Robbins:

In [ ]:
import astropy
import sep
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams

rcParams['figure.figsize'] = [12., 10.]

In [ ]:
fname = 'hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits'
hdu_list = fits.open(fname)
hdu_list.info()
print(hdu_list)

In [ ]:
image_data = hdu_list[0].data
m, s = np.mean(image_data), np.std(image_data)
plt.imshow(image_data, interpolation = 'nearest', cmap='gray', vmin = m-s, vmax = m+s, origin='lower')
plt.colorbar()

Measure a spatially varying background on image


In [ ]:
bkg = sep.Background(image_data, bw=64, bh=64, fw=3, fh=3)
print(bkg.globalback)
print(bkg.globalrms)

In [ ]:
bkg_image = bkg.back() #evaluates background as a 2d array, like the original image
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')

In [ ]:
#background noise as a 2d array
bkg_rms = bkg.rms()
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')

In [ ]:
#subtract the background
data_sub = data-bkg

Object detetction

In [ ]:
thresh = 1.5
objects = sep.extract(data_sub, thresh, err=bkg.globalrms)
#what should thresh be, depends on image?
print('Number of objects detected: ', len(objects))

In [ ]:
from matplotlib.patches import Ellipse

#plot background-subtracted image
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')

#plot an ellipse for each object
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]), width=6*objects['a'][i], height=6*objects['b'][i],
                angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)